In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.stats as sp
import scipy.optimize as sco
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)

In [2]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

def bisection(f, a, b, tol=1e-6):
    c = (a+b)/2
    while(np.abs(b-a)>tol):
        if f(c) == 0:
            return c
        elif f(a)*f(c)<0:
            b=c
        else:
            a=c
        c = (a+b)/2
    return c

def impliedVol(S0, K, tau, r, price, opt_type='c'):
    equation = lambda sigma: BSM(S0,K,tau,r,sigma,opt_type) - price
    return bisection(equation,-1,5)

In [3]:
files = ['20180427', '20180504', '20180511', '20180518', '20180525', '20180601', '20180608', '20180615',
        '20180720', '20180817', '20180921', '20181019', '20181116', '20190118', '20190215', '20190621',
        '20200117']
r = 0.0184 # 3m Treasury 4/30/18
dfs = []
for f in files:
    temp = pd.read_csv('{}.csv'.format(f))
    temp['TTM'] = (pd.to_datetime(temp['T'].apply(lambda x: str(x))) - pd.datetime(2018,4,27)).apply(lambda x: x.days)/365
    dfs.append(temp)
S = dfs[0].S[0]
dfs[0].TTM = 1/365
dfs

[     Unnamed: 0         T       K   P_bid   P_ask   C_bid   C_ask        S  \
 0             0  20180427   910.0    0.00    0.02  607.10  611.80  1517.96   
 1             1  20180427   920.0    0.00    0.03  596.50  601.50  1517.96   
 2             2  20180427   930.0    0.00    3.80  586.70  591.70  1517.96   
 3             3  20180427   940.0    0.00    0.05  576.50  581.50  1517.96   
 4             4  20180427   950.0    0.00    0.05  566.50  571.50  1517.96   
 5             5  20180427   960.0    0.00    1.67  556.40  561.40  1517.96   
 6             6  20180427   970.0    0.00    3.35  546.50  551.50  1517.96   
 7             7  20180427   980.0    0.00    4.25  536.50  541.50  1517.96   
 8             8  20180427   990.0    0.00    0.68  526.55  531.55  1517.96   
 9             9  20180427  1000.0    0.00    0.04  516.55  521.55  1517.96   
 10           10  20180427  1010.0    0.00    0.68  506.55  511.55  1517.96   
 11           11  20180427  1020.0    0.00    0.08  

In [4]:
temp = dfs[0]
tol = 10E-9
def SSVI(df,sig0,s2,c2):
    k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
    return (sig0**2)*(.5*(1 + s2*k) + np.sqrt((.25*(1 + s2*k)**2)) + .5*c2*k**2)
def SSVI_fit(df,IV_col):
    obj = lambda x: np.sum((SSVI(df, x[0], x[1], x[2]) - np.asarray(df[IV_col]))**2)
    cons = ({'type':'ineq','fun': lambda x: x[2]})
    bnds = ((None,None),(None,None),(0,None))
    guess = [.01,.01,.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    return result

SSVI_fit(temp,'Comb_vol')

     fun: 4.527275518449903
     jac: array([-1.52587891e-04, -5.44786453e-05,  7.03334808e-06])
 message: 'Optimization terminated successfully.'
    nfev: 100
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([ 1.11438374e+00, -2.05459548e-02,  2.23160474e+01])

In [15]:
fit1 = SSVI_fit(temp[temp.K < S], 'Comb_vol')['x']
fit2 = SSVI_fit(temp[temp.K > S], 'Comb_vol')['x']

In [16]:
df427 = pd.DataFrame({'K': np.linspace(800,2200,2801), 'TTM': dfs[0].TTM[0]})
df427['Sigma'] = np.append(SSVI(df427[df427.K < S],fit1[0],fit1[1],fit1[2]),
          SSVI(df427[df427.K > S],fit2[0],fit2[1],fit2[2]), axis=0)
df427['C'] = df427.apply(lambda x: BSM(S,x.K,x.TTM,r,x.Sigma), axis=1)
df427

,K,TTM,Sigma,C
0,800.0,0.00274,6.220183,721.288453
1,800.5,0.00274,6.211458,720.777656
2,801.0,0.00274,6.202746,720.266896
3,801.5,0.00274,6.194047,719.756173
4,802.0,0.00274,6.185361,719.245488
5,802.5,0.00274,6.176687,718.734840
6,803.0,0.00274,6.168026,718.224229
7,803.5,0.00274,6.159378,717.713655
8,804.0,0.00274,6.150742,717.203118
9,804.5,0.00274,6.142119,716.692617


In [17]:
trace = go.Scatter(x = df427['K'], 
                   y = (df427.C.shift(-1) - 2*df427.C + df427.C.shift(1))/.5**2, name = 'f(S)')
data = [trace]
layout = go.Layout(title = 'AMZN PDF: 4-27-2018', yaxis = dict(title = 'Probability Density'), xaxis = dict(title = 'S'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [18]:
temp1 = df427[(df427.K < 1517.5)]
temp2 = df427[(df427.K > 1518)]
trace = go.Scatter(x = temp1['K'], 
                   y = (temp1.C.shift(-1) - 2*temp1.C + temp1.C.shift(1))/.5**2, name = 'f(S)')
trace0 = go.Scatter(x = temp2['K'], 
                   y = (temp2.C.shift(-1) - 2*temp2.C + temp2.C.shift(1))/.5**2, name = 'f(S)')
trace1 = go.Scatter(x = 100*[S], y = np.linspace(0,0.005,100), name = 'Underlying')
data = [trace,trace0,trace1]
layout = go.Layout(title = 'AMZN PDF: 4-27-2018', yaxis = dict(title = 'Probability Density'), xaxis = dict(title = 'S'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [14]:
trace0 = go.Scatter(
    x = dfs[0].K, y = 100*dfs[0].Comb_vol, name = 'Implied Volatility'
)
trace1 = go.Scatter(
    x = df427.K, y = 100*df427.Sigma, name = 'SVI Fit'
)
trace2 = go.Scatter(
    x = df427.K, y = (df427.C.shift(-1) - 2*df427.C + df427.C.shift(1))/.5**2, name = 'PDF', yaxis = 'y2'
)

data = [trace0,trace1]
layout = go.Layout(
    title = 'AMZN Implied Volatility: 4-27-2018', 
    yaxis = dict(title = 'Implied Vol (%)'), 
    yaxis2 = dict(title = 'Probability Density', side = 'right'), xaxis = dict(title = 'K'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [9]:
for i in range(len(dfs)):
    temp = SSVI_fit(dfs[i],'Comb_vol')['x']
    trace0 = go.Scatter(
        x = np.exp(np.log(np.asarray(dfs[i].K)/(1517.96*np.exp(r*np.asarray(dfs[i].TTM))))),y =dfs[i].Comb_vol, 
        name = 'Call IV'
    )
    trace1 = go.Scatter(
        x = np.exp(np.log(np.asarray(dfs[i].K)/(1517.96*np.exp(r*np.asarray(dfs[i].TTM))))), 
        y = SSVI(dfs[i],temp[0],temp[1],temp[2]), name = 'Raw SSVI Fit'
    )
    data = [trace0,trace1]
    layout = go.Layout(
        title = 'AMZN Call Raw SSVI Date: {}'.format(files[i]), 
            yaxis = dict(title = 'Implied Volatility'), xaxis = dict(title = 'Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)